In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import sqlite3
import textwrap
import dash
from dash import dcc, html, Input, Output

file_path = r"C:\Users\info\Desktop\project3team11\Resources\final2_cleaned_dataset.csv"
data = pd.read_csv(file_path)

# Define your SQLite database path
db_file = r"C:\Users\info\Desktop\project3team11\Resources\DW_cleaned_data.db"

# Connect to SQLite database (creates the file if it doesn't exist)
conn = sqlite3.connect(db_file)
cursor = conn.cursor()


# Define the table schema based on the dataset columns
cursor.execute('''
CREATE TABLE IF NOT EXISTS health_data (
    Year INTEGER,
    Sex TEXT,
    Age TEXT,
    Grade TEXT,
    Race_Ethnicity TEXT,
    DataSource TEXT,
    Location TEXT,
    LocationID TEXT,
    LocationID_1 INTEGER,
    Geolocation TEXT,
    Topic TEXT,
    TopicID TEXT,
    Question TEXT,
    QuestionID TEXT,
    Value REAL,
    DataValueUnit TEXT,
    DataValueType TEXT,
    DataValueTypeID TEXT,
    LowConfidenceLimit REAL,
    HighConfidenceLimit REAL
)
''')

# Commit the table creation
conn.commit()

# Load data from the DataFrame into the SQLite table
data.to_sql('health_data', conn, if_exists='replace', index=False)

# Confirm data insertion by querying the database
row_count = cursor.execute('SELECT COUNT(*) FROM health_data').fetchone()[0]
print(f"Rows inserted into the database: {row_count}")


Rows inserted into the database: 309215


In [2]:
print(data['Topic'].unique())  # Shows unique topics in the dataset


['Social Determinants of Health' 'Alcohol' 'Tobacco' 'Arthritis'
 'Nutrition, Physical Activity, and Weight Status' 'Health Status'
 'Chronic Obstructive Pulmonary Disease' 'Cardiovascular Disease'
 'Mental Health' 'Cognitive Health and Caregiving' 'Immunization'
 'Diabetes' 'Disability' 'Asthma' 'Oral Health' 'Cancer' 'Sleep'
 'Maternal Health' 'Chronic Kidney Disease']


In [3]:
# Calculate the mode for the 'Topic' column
topic_mode = data['Topic'].mode()[0]

print("The mode for the 'Topic' column is:", topic_mode)

The mode for the 'Topic' column is: Cardiovascular Disease


In [4]:
topic_counts = data['Topic'].value_counts()
topic_counts.plot(kind='bar', figsize=(12, 6))
plt.title('Frequency of Topics')
plt.xlabel('Topic')
plt.ylabel('Count')

plt.xticks(rotation=45, ha='right')

plt.show()

In [5]:
print(data['Question'].unique())  # Shows unique questions related to the data

['High school completion among adults aged 18-24'
 'Unemployment rate among people 16 years and older in the labor force'
 'No broadband internet subscription among households'
 'Living below 150% of the poverty threshold among all people'
 'Per capita alcohol consumption among people aged 14 years and older'
 'Proportion of the population protected by a comprehensive smoke-free policy prohibiting smoking in all indoor areas of workplaces and public places, including restaurants and bars'
 'Have taken an educational class to learn how to manage arthritis symptoms among adults with arthritis'
 'No leisure-time physical activity among adults'
 'Activity limitation due to arthritis among adults with arthritis'
 'Consumed vegetables less than one time daily among adults'
 'Obesity among adults' 'Frequent physical distress among adults'
 'Work limitation due to arthritis among adults aged 18-64 years with arthritis'
 'Chronic obstructive pulmonary disease among adults'
 'Binge drinking prev

In [6]:
print(data['QuestionID'].unique())  # Shows unique questions related to the data

['SDH02' 'SDH08' 'SDH03' 'SDH01' 'ALC08' 'TOB07' 'ART07' 'NPW06' 'ART02'
 'NPW04' 'NPW14' 'HEA03' 'ART04' 'COPD01' 'ALC06' 'HEA05' 'SDH12' 'CVD02'
 'MEN04' 'CHC01' 'ALC04' 'HEA02' 'NPW09' 'TOB04' 'CHC03' 'CVD04' 'IMM03'
 'TOB06' 'SDH09' 'MEN02' 'CHC02' 'ART01' 'DIA01' 'IMM04' 'IMM01' 'CVD01'
 'NPW02' 'COPD02' 'HEA04' 'MEN05' 'ALC05' 'ART03' 'DIS01' 'CHC04' 'CVD03'
 'ART06' 'ART05' 'IMM02' 'AST02' 'HEA01' 'ORH01' 'CAN06' 'SLP03' 'SDH07'
 'SDH06' 'ORH03' 'ORH06' 'SDH05' 'CAN09' 'ORH02' 'CAN10' 'COPD03' 'CVD06'
 'COPD04' 'SDH04' 'NPW10' 'NPW11' 'SLP01' 'ORH05' 'NPW07' 'ORH04' 'CVD09'
 'COPD06' 'ALC09' 'CVD07' 'DIA04' 'AST01' 'DIA03' 'CVD08' 'COPD05' 'HEA06'
 'DIA02' 'SDH10' 'SDH11' 'TOB05' 'MAT01' 'ORH07' 'MEN03' 'CAN08' 'CAN07'
 'CAN05' 'CAN03' 'CAN01' 'CAN04' 'CAN02' 'CKD01' 'NPW12' 'ALC01' 'NPW05'
 'SLP02' 'ALC07' 'TOB01' 'NPW03' 'NPW08' 'TOB02' 'TOB03' 'NPW13' 'NPW01'
 'MEN01']


In [7]:
# Clean the dataset (ensure numeric values in Value column)
data['Value'] = pd.to_numeric(data['Value'], errors='coerce')  # Convert Value column to numeric
data = data.dropna(subset=['Value'])  # Remove rows with missing values in Value

# Aggregate data to remove duplicates
aggregated_data = data.groupby(['Year', 'Topic'], as_index=False).agg({'Value': 'mean'})
print("Aggregated Data:")
print(aggregated_data.head())
print(aggregated_data.shape)

# Pivot the dataset
pivoted_data = aggregated_data.pivot(index='Year', columns='Topic', values='Value')
print("Pivoted Data:")
print(pivoted_data.head())
print(pivoted_data.shape)

# Check for missing values in pivoted data
pivoted_data = pivoted_data.fillna(0)  # Fill missing values with 0

# Compute correlations
correlations = pivoted_data.corr()
print("Correlation Matrix:")
print(correlations)



Aggregated Data:
   Year                   Topic        Value
0  2019                 Alcohol    83.206742
1  2019               Arthritis    33.048481
2  2019                  Asthma    25.138099
3  2019                  Cancer  2202.719539
4  2019  Cardiovascular Disease  1638.525750
(67, 3)
Pivoted Data:
Topic     Alcohol  Arthritis     Asthma       Cancer  Cardiovascular Disease  \
Year                                                                           
2019    83.206742  33.048481  25.138099  2202.719539             1638.525750   
2020   117.925678        NaN  27.684740  1798.138743             2685.483736   
2021   103.008165  31.644240  25.200346          NaN             1601.739159   
2022     9.070506  26.764120  11.271670    60.236407                     NaN   

Topic  Chronic Kidney Disease  Chronic Obstructive Pulmonary Disease  \
Year                                                                   
2019              2702.197115                            3923.5995

In [8]:
# Check the schema of the table
query = "PRAGMA table_info(health_data)"
table_info = conn.execute(query).fetchall()

# Print the table schema
for column in table_info:
    print(column)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Sex', 'TEXT', 0, None, 0)
(2, 'Age', 'TEXT', 0, None, 0)
(3, 'Grade', 'TEXT', 0, None, 0)
(4, 'Race/Ethnicity', 'TEXT', 0, None, 0)
(5, 'DataSource', 'TEXT', 0, None, 0)
(6, 'Location', 'TEXT', 0, None, 0)
(7, 'LocationID', 'TEXT', 0, None, 0)
(8, 'LocationID.1', 'INTEGER', 0, None, 0)
(9, 'Geolocation', 'TEXT', 0, None, 0)
(10, 'Longitude', 'REAL', 0, None, 0)
(11, 'Latitude', 'REAL', 0, None, 0)
(12, 'Topic', 'TEXT', 0, None, 0)
(13, 'TopicID', 'TEXT', 0, None, 0)
(14, 'Question', 'TEXT', 0, None, 0)
(15, 'QuestionID', 'TEXT', 0, None, 0)
(16, 'Value', 'REAL', 0, None, 0)
(17, 'DataValueUnit', 'TEXT', 0, None, 0)
(18, 'DataValueType', 'TEXT', 0, None, 0)
(19, 'DataValueTypeID', 'TEXT', 0, None, 0)
(20, 'LowConfidenceLimit', 'REAL', 0, None, 0)
(21, 'HighConfidenceLimit', 'REAL', 0, None, 0)


In [9]:
import sqlite3

# Define the database file path
db_file = r"C:\Users\info\Desktop\project3team11\Resources\DW_cleaned_data.db"

# Connect to the database and fetch table names
conn = sqlite3.connect(db_file)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
conn.close()

print("Available Tables:", tables)


Available Tables: [('health_data',)]


DASH LIFESTYLE INTERACTIVE VISUALS

In [10]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd
import sqlite3
import textwrap

# Define your SQLite database path
db_file = r"C:\Users\info\Desktop\project3team11\Resources\DW_cleaned_data.db"

# Initialize Dash app
app = dash.Dash(__name__)
app.title = "Health Measures Dashboard"

# Connect to SQLite and load unique options for dropdowns
conn = sqlite3.connect(db_file)
topics = pd.read_sql_query('SELECT DISTINCT Topic FROM health_data', conn)['Topic'].tolist()
locations = pd.read_sql_query('SELECT DISTINCT Location FROM health_data', conn)['Location'].tolist()
years = pd.read_sql_query('SELECT DISTINCT Year FROM health_data', conn)['Year'].sort_values().tolist()
sexes = ['All'] + pd.read_sql_query('SELECT DISTINCT Sex FROM health_data', conn)['Sex'].tolist()
ages = ['All'] + pd.read_sql_query('SELECT DISTINCT Age FROM health_data', conn)['Age'].tolist()
races = ['All'] + pd.read_sql_query('SELECT DISTINCT "Race/Ethnicity" FROM health_data', conn)['Race/Ethnicity'].tolist()
conn.close()

# Define the app layout
app.layout = html.Div(style={"backgroundColor": "#f9f9f9", "fontFamily": "Arial, sans-serif", "padding": "20px"}, children=[
    # App Title
    html.H1("Interactive Health Measures Dashboard", style={
        "textAlign": "center",
        "color": "#2c3e50",
        "fontWeight": "bold",
        "fontSize": "36px",
        "marginBottom": "10px"
    }),

    # Introduction Section
    html.Div([
        html.H3("Project 11 Team 3", style={"textAlign": "center", "color": "#2c3e50", "fontWeight": "bold"}),
        html.P(
            "Healthy Living, Diverse Challenges: Exploring Health Trends by Topic and Region",
            style={"textAlign": "center", "color": "#34495e", "fontSize": "18px", "marginBottom": "10px"}
        ),
        html.P(
            "This dashboard empowers users to uncover key health trends by exploring topics, regions," 
            "and demographics. Dive into interactive visualizations to identify patterns, reveal insights," 
            "and better understand the relationships between chronic diseases, health risks," 
            "and social determinants. Our mission: Make data-driven decision-making accessible" 
            "and impactful for all.",
            style={"textAlign": "center", "color": "#34495e", "fontSize": "16px", "marginBottom": "20px"}
        )
    ], style={"marginBottom": "30px"}),

    # Filters Section
    html.Div([
        html.Label("Select Topic:", style={"fontWeight": "bold", "color": "#2c3e50"}),
        dcc.Dropdown(id="topic-dropdown", options=[{"label": t, "value": t} for t in topics], value=topics[0]),
        html.Label("Select Location:", style={"fontWeight": "bold", "color": "#2c3e50"}),
        dcc.Dropdown(id="location-dropdown", options=[{"label": l, "value": l} for l in locations], value=locations[0]),
        html.Label("Select Year:", style={"fontWeight": "bold", "color": "#2c3e50"}),
        dcc.Slider(id="year-slider", min=min(years), max=max(years), step=1, value=min(years),
                   marks={year: str(year) for year in years})
    ], style={"width": "50%", "margin": "auto", "padding": "20px", "border": "1px solid #ddd", "borderRadius": "10px", "backgroundColor": "#ffffff"}),

    # Demographic Filters Section
    html.Div([
        html.Label("Select Sex:", style={"fontWeight": "bold", "color": "#2c3e50"}),
        dcc.Dropdown(id="sex-dropdown", options=[{"label": s, "value": s} for s in sexes], value='All'),
        html.Label("Select Age:", style={"fontWeight": "bold", "color": "#2c3e50"}),
        dcc.Dropdown(id="age-dropdown", options=[{"label": a, "value": a} for a in ages], value='All'),
        html.Label("Select Race/Ethnicity:", style={"fontWeight": "bold", "color": "#2c3e50"}),
        dcc.Dropdown(id="race-dropdown", options=[{"label": r, "value": r} for r in races], value='All')
    ], style={"width": "50%", "margin": "auto", "padding": "20px", "border": "1px solid #ddd", "borderRadius": "10px", "backgroundColor": "#ffffff", "marginTop": "20px"}),

    # Graphs Section
    dcc.Graph(id="bar-chart"),
    dcc.Graph(id="choropleth-map"),
    dcc.Graph(id="line-chart"),
    dcc.Graph(id="scatter-plot"),
    dcc.Graph(id="top-questions-chart")
])

# Define callback to update visuals
@app.callback(
    Output("bar-chart", "figure"),
    Output("choropleth-map", "figure"),
    Output("line-chart", "figure"),
    Output("scatter-plot", "figure"),
    Output("top-questions-chart", "figure"),
    Input("topic-dropdown", "value"),
    Input("location-dropdown", "value"),
    Input("year-slider", "value"),
    Input("sex-dropdown", "value"),
    Input("age-dropdown", "value"),
    Input("race-dropdown", "value")
)
def update_visuals(selected_topic, selected_location, selected_year, selected_sex, selected_age, selected_race):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_file)

    # Build a base query
    query = f'''
    SELECT * FROM health_data
    WHERE Topic = "{selected_topic}" 
    AND Location = "{selected_location}" 
    AND Year = {selected_year}
    '''
    
    # Add demographic filters to the query
    if selected_sex != 'All':
        query += f' AND Sex = "{selected_sex}"'
    if selected_age != 'All':
        query += f' AND Age = "{selected_age}"'
    if selected_race != 'All':
        query += f' AND "Race/Ethnicity" = "{selected_race}"'

    # Query the database
    data = pd.read_sql_query(query, conn)

    # Query for trends over time
    trend_query = f'''
    SELECT Year, AVG(Value) as AvgValue FROM health_data
    WHERE Topic = "{selected_topic}" AND Location = "{selected_location}"
    '''
    if selected_sex != 'All':
        trend_query += f' AND Sex = "{selected_sex}"'
    if selected_age != 'All':
        trend_query += f' AND Age = "{selected_age}"'
    if selected_race != 'All':
        trend_query += f' AND "Race/Ethnicity" = "{selected_race}"'
    trend_query += " GROUP BY Year"
    trend_data = pd.read_sql_query(trend_query, conn)

    # Query for top 10 questions
    top_questions_query = f'''
    SELECT Question, AVG(Value) as AvgValue FROM health_data
    WHERE Topic = "{selected_topic}"
    GROUP BY Question
    ORDER BY AvgValue DESC
    LIMIT 10
    '''
    top_questions_data = pd.read_sql_query(top_questions_query, conn)

    # Add line breaks to long question labels
    top_questions_data['Question'] = top_questions_data['Question'].apply(
        lambda q: "<br>".join(textwrap.wrap(q, width=40))
    )

    conn.close()

    # Create Visuals
    # Bar Chart
    bar_fig = px.bar(data, x="Question", y="Value", title=f"Bar Chart for {selected_topic} in {selected_location} ({selected_year})")
    bar_fig.update_layout(
        title={"x": 0.5, "font": {"size": 24, "color": "#2c3e50"}},
        plot_bgcolor="#f9f9f9", paper_bgcolor="#ffffff",
        font={"family": "Arial", "color": "#2c3e50"}
    )

    # Choropleth Map
    map_fig = px.choropleth(data, locations="LocationID", locationmode="USA-states", color="Value",
                            title="Geographic Distribution of Measures")
    map_fig.update_layout(
        title={"x": 0.5, "font": {"size": 24, "color": "#2c3e50"}},
        geo=dict(bgcolor="#f9f9f9"),
        paper_bgcolor="#ffffff",
        font={"family": "Arial", "color": "#2c3e50"}
    )

    # Line Chart
    line_fig = px.line(trend_data, x="Year", y="AvgValue", title=f"Trends for {selected_topic} in {selected_location}")
    line_fig.update_layout(
        title={"x": 0.5, "font": {"size": 24, "color": "#2c3e50"}},
        plot_bgcolor="#f9f9f9", paper_bgcolor="#ffffff",
        font={"family": "Arial", "color": "#2c3e50"}
    )

    # Scatterplot
    scatter_fig = px.scatter(data, x="LowConfidenceLimit", y="HighConfidenceLimit",
                              title="Correlation Between Confidence Limits")
    scatter_fig.update_layout(
        title={"x": 0.5, "font": {"size": 24, "color": "#2c3e50"}},
        plot_bgcolor="#f9f9f9", paper_bgcolor="#ffffff",
        font={"family": "Arial", "color": "#2c3e50"}
    )

    # Top 10 Questions Chart with Multi-Line Labels
    top_questions_fig = px.bar(
        top_questions_data,
        x="Question",
        y="AvgValue",
        title=f"Top 10 Questions for {selected_topic}",
        labels={"Question": "Questions", "AvgValue": "Average Value"}
    )
    top_questions_fig.update_layout(
        title={"x": 0.5, "font": {"size": 24, "color": "#2c3e50"}},
        xaxis={"title": "Questions", "titlefont": {"size": 18, "color": "#34495e"}},
        yaxis={"title": "Average Value", "titlefont": {"size": 18, "color": "#34495e"}},
        plot_bgcolor="#f9f9f9",
        paper_bgcolor="#ffffff",
        font={"family": "Arial", "color": "#2c3e50"}
    )

    return bar_fig, map_fig, line_fig, scatter_fig, top_questions_fig

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)
